In [ ]:
%load_ext dotenv
%dotenv
import openai
import re
import requests
import sys
from num2words import num2words
import os
from pprint import pprint
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast

API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
RESOURCE_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"
print(openai.api_base)
url = openai.api_base + "/openai/deployments?api-version=2022-12-01"

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

In [ ]:
def normalize_text(s, sep_token = " \n "):
    print(s)
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

In [ ]:
def generateTextOpenAI(prompt):
    response = openai.Completion.create(engine="davinchi-003", prompt=prompt, max_tokens=1024)
    return response['choices'][0]['text']

In [ ]:
with open("data/openning bank account.txt", "r") as file:
    doc = file.read()


In [ ]:

# FAQ Example

prompt = f"""
            {normalize_text(doc)}
            divide the text into sections and put it in json format with a list of dictionaries with the following format:
            [
                {{"<section 1 title>":"<the text of section 1>"}},
                {{"<section 2 title>":"<the text of section 2>"}}
            ]
            """
print(prompt)    
text = generateTextOpenAI(prompt=prompt) 
    

In [ ]:
import json
sections = json.loads(normalize_text(text))
pprint(sections)

In [ ]:
FAQs = []
for i in sections:
    content = list(i.values())[0]
    prompt = f"""
        {content}
        
        generate FAQ questions and answers into json text with the following format (write only the json text):
        [
            {{"<question 1>":"<answer 1>"}},
            {{"<question 2>":"<answer 2>"}}
        ]
    """
    print(prompt)
    text = generateTextOpenAI(prompt=prompt)
    try:
        FAQ_dict = json.loads(normalize_text(text))
        FAQs.append({list(i.keys())[0]: FAQ_dict})
    except Exception as e:
        print(e)
        print("Error in parsing the json text")

pprint(FAQs)

In [ ]:
print("")
for FAQ in FAQs:
    print(list(FAQ.keys())[0])
    for i in list(FAQ.values())[0]:
        print("")
        print(f"question: {list(i.keys())[0]}")
        print(f"answer:   {list(i.values())[0]}")
        
        
    